In [ ]:
# 'wiki' index 생성 - 1536 dim

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
index_name = "wiki"
for idx in pc.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

In [4]:
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-506yx3r.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [5]:
# 'wiki' 인덱스를 가져옵니다.
index = pc.Index(index_name)

index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# data load

In [6]:
%pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [9]:
from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

In [10]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [11]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [12]:
len(data)

100

In [13]:
for rec in data:
  if len(rec['text'])>35000:
    print('**')

**
**


In [14]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001CD5CEBBA40>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001CD5EA37590>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
  chunk_size=400,
  chunk_overlap =20,
  length_function = len, \
  separators=["\n\n", "\n", " ", ""]  #자르는 순서 문자
)
splitter

In [19]:
import json
def calculate_metadata_size(metadata):
  return len(json.dumps(metadata, ensure_ascii=False).encode('utf-8'))

In [22]:
from uuid import uuid4
import time
texts = []
metas = []
count = 0
batch_size = 20

In [23]:
for i, sample in enumerate(data):
  full_text = sample['text'] #전문
  
  #메타데이터 구성
  metadata = {
    "wiki_id": str(sample['id']),
    "url": sample['url'],
    "title": sample['title'] #[:100]
  }
  
  # 스플릿터로 자르기
  chunks = splitter.split_text(full_text)
  #print(len(chunks))
  
  for i, chunk in enumerate(chunks):
    record = {
      'chunk_id':i,
      'full_text':full_text,
      **metadata
    }
    metadata_size = calculate_metadata_size(record)
    if metadata_size > 35000:
      continue
    
    texts.append(chunk)
    metas.append(record)
    count += 1
    
    # 청크 배치의 임베딩 -> 적재
    if count % batch_size == 0:
      ids = [str(uuid4()) for _ in range(len(texts))]
      embeddings = embedding.embed_documents(texts) #20개 청크의 임베딩 수행
      index.upsert(
        vectors=zip(ids, embeddings, metas),
        namespace='wiki-ns1'
      )
      texts=[]
      metas=[]
      time.sleep(1)

In [25]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 1160}},
 'total_vector_count': 1160,
 'vector_type': 'dense'}

# 질의 임베딩

In [26]:
question = ["벨기에는 어디에 있나요?"]
emb_question = embedding.embed_documents(question)
emb_question

[[0.0029637617990374565,
  0.0386190339922905,
  0.08355793356895447,
  0.011749008670449257,
  -0.01028568483889103,
  -0.015439130365848541,
  0.011886858381330967,
  0.01581026427447796,
  -0.02797282114624977,
  -0.021971071138978004,
  -0.01926710270345211,
  0.058151230216026306,
  0.042224325239658356,
  -0.03075101599097252,
  0.0014792298898100853,
  0.005582899786531925,
  -0.0692640095949173,
  0.016764605417847633,
  0.02687002532184124,
  0.02194986306130886,
  -0.02337077260017395,
  -0.022904204204678535,
  -0.032256755977869034,
  0.04869264364242554,
  0.042542438954114914,
  0.007162865716964006,
  0.015576980076730251,
  0.0003840563294943422,
  0.006823544390499592,
  -0.03618016093969345,
  0.02264971286058426,
  -0.03255366161465645,
  0.02139846608042717,
  -0.05034683644771576,
  0.014198485761880875,
  0.00985092855989933,
  0.04504493623971939,
  -0.04411180317401886,
  0.017856797203421593,
  -0.023498017340898514,
  -0.006081278435885906,
  0.005150794982910

# Retrieval

In [28]:
# 파인콘 질의 실행
q_result = index.query(
  namespace='wiki-ns1',
  vector=emb_question,
  top_k=5,
  include_vector = False,
  include_metadata=True
)
q_result

{'matches': [{'id': 'eba1897c-7877-4b38-bda3-33d7dac5962a',
              'metadata': {'chunk_id': 11.0,
                           'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 '
                                        '없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. '
                                        '통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 '
                                        '로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, '
                                        '열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 '
                                        '루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 '
                                        '라틴 대문자 S를 쓴다.\n'
                                        '\n'
                                        '정의 \n'
                                        '엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 '
                                        '정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. '
                                        '고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 '
                                      

In [30]:
result_ids = [ r.id for r in q_result.matches]
result_ids

['eba1897c-7877-4b38-bda3-33d7dac5962a',
 'f7afb2e3-5336-438b-bb9b-956063456343',
 'aff8b363-5e2b-4599-bd25-cf83d59f8752',
 '3c996bc6-b210-43b8-9c49-fc7530e9f7bf',
 '7e95d3ac-8f14-4a4f-b4e2-b384d8b79ef1']

In [33]:
for r in q_result.matches:
  print(r.score, r.metadata)

0.306882858 {'chunk_id': 11.0, 'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하는데 있어서 점점 신뢰할 수 없는 예측 기술이 되어감으로 인하여 필수적인 것이 되었다.\n\n열역학적 정의 \n고전적 열역학에서는 엔트로피 S의 절대적 값은 정의할 수 없고, 대신 그 상대적 변화만 정의한다. 열적 평형을 이뤄 온도가 인 계에 열 를 가하였다고 하자. 이 경우 엔트로피의 증가

# 랭체인의 api 로 결과 비교하기

In [34]:
question

['벨기에는 어디에 있나요?']

In [35]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

[Document(id='eba1897c-7877-4b38-bda3-33d7dac5962a', metadata={'chunk_id': 11.0, 'title': '엔트로피', 'url': 'https://ko.wikipedia.org/wiki/%EC%97%94%ED%8A%B8%EB%A1%9C%ED%94%BC', 'wiki_id': '158'}, page_content='엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하

#질문의 응답 결과가 동일한지 검증합니다.

In [ ]:
question=[
  "",
  "",
  "",
]

# Augmentation

In [40]:
#retrieval
country = '벨기에'
query = f"{country}의 여행을 위한 관광지와 문화"
docs = vector_store.similarity_search(query=query, k=5, namespace='wiki-ns1')

In [41]:
#augmentation context
context = ""
for result in docs:
  context += result.page_content

In [42]:
context

'아쿠타가와 류노스케(, 1892년 3월 1일~1927년 7월 24일)는 일본의 소설가이다. 호는 징강당주인(澄江堂主人)이며 하이쿠 작가로서의 호는 가키(我鬼)이다.\n\n그의 작품은 대부분이 단편 소설이다. 「참마죽」, 「덤불 속」, 「지옥변」 등 주로 일본의 《곤자쿠 이야기집》·《우지슈이 이야기》 등 전통적인 고전들에서 제재를 취하였다. 또한 「거미줄(원제: 蜘蛛の糸)」, 「두자춘(杜子春)」 등 어린이를 위한 작품도 남겼으며, 예수를 학대한 유대인이 예수가 세상에 다시 올 때까지 방황한다는 상상력을 발휘한 「방황하는 유대인」도 있다.\n\n생애\n\n유년 시절 \n1892년(메이지 25년) 3월 1일 도쿄에서 우유 판매업자였던 아버지 니하라 도시조(新原敏三)와 어머니 후쿠(フク) 사이의 아들로 태어났다(아쿠타가와라는 성은 원래 그의 어머니쪽 성씨였다). 이때 태어난 시간이 공교롭게도 진년(辰年) 진월(辰月) 진일(辰日) 진시(辰時)였기 때문에 \'용(龍)\' 자를 이름에 넣어 류노스케(龍之介)라 짓게 되었다고 전하나, 실제 그가 태어난 1892년 3월 1일은 간지로는 임진년·임인월·임진일에 해당하며, 출생 시각에 대해서는 자료가 없기 때문에 확실한 것이 없다. 이름도 호적상으로는 \'龍之介\'이지만 그가 양자로 들어갔던 아쿠타가와 집안이나 졸업한 학교의 명단 등의 문서에는 \'龍之助\'로 되어 있다(아쿠타가와 자신은 \'龍之助\' 표기를 싫어했다).\n\n가족 \n원래 류노스케의 위로는 하쓰(はつ)와 히사(ひさ)라는 두 명의 누나가 있었는데, 큰누나였던 하쓰는 류노스케가 태어나기 1년 전에 여섯 살의 나이로 요절했고, 어머니는 그 충격으로 정신장애를 겪어서 류노스케를 양육할 수 없었다. 생후 7개월 된 류노스케는 도쿄시 혼죠구 고이즈미쵸에 있던 외가 아쿠타가와 집안에 맡겨졌고, 백모 후키(フキ)가 양육을 맡았다. 11살 때인 1902년에 어머니가 끝내 사망하자 이듬해에 그는 외삼촌으로 도쿄시의 토목과장을 지내기도 했던 아쿠타가와 미치아키(芥川道章)의 양자가

In [37]:
# generation
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001CD5F5A8DA0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001CD5F99BA10>, root_client=<openai.OpenAI object at 0x000001CD5F06A270>, root_async_client=<openai.AsyncOpenAI object at 0x000001CD5F998FE0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [38]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
  [
    ('system', """당신은 베테랑 여행 가이드입니다. 
                  고객 맞춤 여행일정 수립을 도와드립니다. 
                  제한된 정보라도 최대한 활용해서 도움을 주세요."""),
    ('human', """저는 {country}로 여행을 계획중입니다.  
                아래 context에서 {country}관련 정보를 찾아 여행 조언을 해주세요.
                만약, 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.
                context :
                {context}""")    
  ]
)
chat_template

ChatPromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 베테랑 여행 가이드입니다. \n                  고객 맞춤 여행일정 수립을 도와드립니다. \n                  제한된 정보라도 최대한 활용해서 도움을 주세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, template='저는 {country}로 여행을 계획중입니다.  \n                아래 context에서 {country}관련 정보를 찾아 여행 조언을 해주세요.\n                만약, 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.\n                context :\n                {context}'), additional_kwargs={})])

In [43]:
prompt = chat_template.format_messages(country=country, context=context)
print(prompt)

[SystemMessage(content='당신은 베테랑 여행 가이드입니다. \n                  고객 맞춤 여행일정 수립을 도와드립니다. \n                  제한된 정보라도 최대한 활용해서 도움을 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='저는 벨기에로 여행을 계획중입니다.  \n                아래 context에서 벨기에관련 정보를 찾아 여행 조언을 해주세요.\n                만약, 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.\n                context :\n                아쿠타가와 류노스케(, 1892년 3월 1일~1927년 7월 24일)는 일본의 소설가이다. 호는 징강당주인(澄江堂主人)이며 하이쿠 작가로서의 호는 가키(我鬼)이다.\n\n그의 작품은 대부분이 단편 소설이다. 「참마죽」, 「덤불 속」, 「지옥변」 등 주로 일본의 《곤자쿠 이야기집》·《우지슈이 이야기》 등 전통적인 고전들에서 제재를 취하였다. 또한 「거미줄(원제: 蜘蛛の糸)」, 「두자춘(杜子春)」 등 어린이를 위한 작품도 남겼으며, 예수를 학대한 유대인이 예수가 세상에 다시 올 때까지 방황한다는 상상력을 발휘한 「방황하는 유대인」도 있다.\n\n생애\n\n유년 시절 \n1892년(메이지 25년) 3월 1일 도쿄에서 우유 판매업자였던 아버지 니하라 도시조(新原敏三)와 어머니 후쿠(フク) 사이의 아들로 태어났다(아쿠타가와라는 성은 원래 그의 어머니쪽 성씨였다). 이때 태어난 시간이 공교롭게도 진년(辰年) 진월(辰月) 진일(辰日) 진시(辰時)였기 때문에 \'용(龍)\' 자를 이름에 넣어 류노스케(龍之介)라 짓게 되었다고 전하나, 실제 그가 태어난 1892년 3월 1일은 간지로는 임진년·임인월·임진일에 해당하며, 출생 시각에 대해서는 자료가 없기 때문에 확실한 것이 없다. 이름도 호적상으로는 \'龍之介\'

In [44]:
response = llm.invoke(prompt)
response.content

'벨기에 여행을 계획하고 계신다니 정말 흥미롭습니다! 벨기에는 멋진 도시와 맛있는 음식, 풍부한 역사, 그리고 독특한 문화가 가득하죠. 주어진 정보를 통해 벨기에의 관련 정보와 수도 브뤼셀을 중심으로 여행 일정을 제안해드릴게요.\n\n### 1일차: 브뤼셀 탐방\n- **그랑 플라스**: 브뤼셀의 중심 광장으로, 유네스코 세계문화유산에 등록되어 있습니다. 아름다운 건축물과 분수대가 매력적입니다.\n- **시청사**: 그랑 플라스 바로 옆에 위치하고 있으며, 고딕 양식의 건축물로 유명합니다.\n- **맨켄피스 (Manneken Pis)**: 브뤼셀의 상징적인 동상인 소년 소변하는 모습의 조각상을 보러 가세요.\n- **벨기에 초콜릿 가게 투어**: 벨기에는 세계적인 초콜릿으로 유명하니, 유명 초콜릿 가게에서 샘플을 맛보는 것이 좋습니다.\n\n### 2일차: 브뤼셀 근교 여행\n- **브뤼헤(Bruges)**: 중세 시대의 아름다움이 남아있는 도시로, 운하와 자갈길이 매력적입니다. 자전거를 대여해서 도시를 구경해보세요.\n- **겐트(Ghent)**: 역사적인 건축물과 예술이 많은 도시입니다. 겐트 성과 성 바브 대성당을 꼭 방문해보세요.\n\n### 3일차: 앤트워프 방문\n- **앤트워프 대성당**: 이곳은 르네상스 양식의 아름다운 성당이며, 내부에는 루벤스의 작품이 있습니다.\n- **다이아몬드 지구**: 앤트워프는 다이아몬드로 유명합니다. 다이아몬드 상점들을 돌아보며 정보를 알아보세요.\n- **레드 옥탑**: 앤트워프의 랜드마크 중 하나로, 멋진 경관을 즐길 수 있는 곳입니다.\n\n### 4일차: 벨기에 맥주 체험\n- **브뤼셀의 맥주 양조장**: 벨기에는 세계적인 맥주로 유명하므로, 맥주 양조장을 방문해서 시음 체험을 해보세요.\n- **벨기에 전통 음식 즐기기**: 뮈스리(감자튀김)와 매동고기(스튜) 등의 전통 음식을 맛보세요.\n\n### 5일차: 역사와 문화\n- **전쟁 박물관**: 벨기에의 다양한 전쟁 역사와 문화를 배울 수 있는 기회를 

In [45]:
# 벡터 검색 + 키워드 필터링
def enhanced_search(country, vector_store):
    # 1단계: 벡터 검색
    vector_results = vector_store.similarity_search(
        query=f"{country} 여행 관광", 
        k=10, 
        namespace="wiki-ns1"
    )
    
    # 2단계: 국가명이 포함된 결과 우선순위
    country_related = []
    other_results = []
    
    for result in vector_results:
        if country.lower() in result.page_content.lower() or \
           "벨기에" in result.page_content or \
           "브뤼셀" in result.page_content:
            country_related.append(result)
        else:
            other_results.append(result)
    
    # 관련성 높은 결과 우선 반환
    final_results = (country_related + other_results)[:5]
    return final_results

results = enhanced_search("벨기에", vector_store)

In [46]:
results

[Document(id='7e95d3ac-8f14-4a4f-b4e2-b384d8b79ef1', metadata={'chunk_id': 5.0, 'title': '나라 이름순 수도 목록', 'url': 'https://ko.wikipedia.org/wiki/%EB%82%98%EB%9D%BC%20%EC%9D%B4%EB%A6%84%EC%88%9C%20%EC%88%98%EB%8F%84%20%EB%AA%A9%EB%A1%9D', 'wiki_id': '79'}, page_content='국\n\nㄱ \n\n 가나 - 아크라 \n 가봉 - 리브르빌\n 가이아나 - 조지타운\n 감비아 - 반줄\n 괌 - 하갓냐\n 과테말라 - 과테말라\n 그레나다 - 세인트조지스\n 그리스 - 아테네\n 그린랜드 - 누크\n 기니 - 코나크리\n 기니비사우 - 비사우\n\nㄴ \n\n 나미비아 - 빈트후크\n 나우루 - 야렌\n 나이지리아 - 아부자\n 남수단 - 주바\n 남아프리카 공화국 - 프리토리아(행정), 블룸폰테인(사법), 케이프타운(입법)\n 남오세티야 - 츠힌발리\n 네덜란드 - 암스테르담(헌법 상), 헤이그(행정)\n 네팔 - 카트만두\n 노르웨이 - 오슬로\n 뉴질랜드 - 웰링턴\n 니우에 - 알로피\n 니제르 - 니아메\n 니카라과 - 마나과\n\nㄷ \n\n 대한민국 - 서울특별시(사실상), 세종특별자치시(행정)\n 덴마크 - 코펜하겐\n 도미니카 연방 - 로조\n 도미니카 공화국 - 산토도밍고\n 독일 - 베를린\n 동티모르 - 딜리\n\nㄹ \n\n 라오스 - 비엔티안\n 라이베리아 - 몬로비아\n 라트비아 - 리가\n 러시아 - 모스크바\n 레바논 - 베이루트\n 레소토 - 마세루\n 루마니아 - 부쿠레슈티\n 룩셈부르크 - 룩셈부르크\n 르완다 - 키갈리\n 리비아 - 트리폴리\n 리투아니아 - 빌뉴스\n 리히텐슈타인 - 파두츠\n\nㅁ \n\n 마다가스카르 - 안타나나리보\n 마셜 제도 - 마주로\n 마케도니아 공화국 - 스코페\n 말라위 - 릴롱궤\n 말레이시아